### Laboration Statistiska Metoder
_Björn Winterfjord_

Vi börjar med att importera nödvändiga bibliotek, inklusive filen <span style = "color:green">LR.py</span>, där våra funktioner för linjär regression finns. Därefter läser vi in datafilen <span style = "color:green">Small-diameter-flow.csv</span>.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import LR

data_path = "Small-diameter-flow.csv"
df = pd.read_csv(data_path, index_col=0)


Vi låter responsvektorn y vara kolumnen <span style = "color:green">Flow</span>, och vi ska undersöka om det finns något samband mellan denna och de tre variablerna <span style = "color:green">Kinematic</span>, <span style = "color:green">Geometric</span> och <span style = "color:green">Inertial</span>. Vi sparar dessa tre kolumner i vår designmatris <span style = "color:green">X</span>, tillsammans med en kolumn med 1:or.

Vi tar alltså inte med kolumnen <span style = "color:green">Observer</span> i nuläget, men kommer senare att underöka om det finns något observatörsbias i experimentet.

In [2]:
y = df["Flow"]
X = np.column_stack([np.ones(y.shape[0]), df['Kinematic'], df['Geometric'], df['Inertial']])

Nu utför vi vår linjära regression genom att anropa funktionen <span style = "color:green">fit</span> från <span style = "color:green">LR.py</span>. Vi söker ett linjärt samband på formen
$$
y=\beta_0 + \beta_1 [Kinematic] + \beta_2 [Geometric] + \beta_3[Inertial]
$$
där $\beta_0$ är interceptet och $\beta_1$, $\beta_2$ och $\beta_3$ är parametrar för de olika x-variablerna. Vi kommer senare att utvärdera rimligheten i denna modell.

In [3]:

m = LR.LinearRegression()
b = m.fit(X, y)
print(f"Vi får β0 = {b[0]}, β1 = {b[1]}, β2 = {b[2]}, β3 = {b[3]}")


Vi får β0 = -2.5597931356705477, β1 = 0.8687151918648803, β2 = 3.610418179476874, β3 = -0.7536877232172592


In [3]:
d = m.dimension(b)
print(d)

3


In [4]:
n = m.sample_size(y)
print(n)

198


In [5]:
SSE = m.SSE(y, X, b)
print(SSE)


1.2238849845911814


In [6]:
var = m.variance(SSE, n, d)
print(var)


0.00630868548758341


In [7]:
S = m.deviation(var)
print(S)

0.07942723391622933


In [8]:
Syy = m.Syy(n, y)
print(Syy)


425.14419302220614


In [9]:
SSR = m.SSR(Syy, SSE)
print(SSR)


423.920308037615


In [10]:
F = m.Fstatistic(SSR, d, var)
print(F)


22398.7658956827


In [11]:
Rsq = m.Rsq(SSR, Syy)
print(Rsq)


0.9971212473210772


In [12]:
# Dependency check

print(m.Pearsonr(X, 1, 2))
print(m.Pearsonr(X, 1, 3))
print(m.Pearsonr(X, 2, 3))

PearsonRResult(statistic=0.8631350761065918, pvalue=4.5604633624399433e-60)
PearsonRResult(statistic=0.9686707504997812, pvalue=1.588545639896567e-120)
PearsonRResult(statistic=0.9183300308547002, pvalue=7.951572627158216e-81)


In [13]:
# Beräkna varians/kovarians-matris
c = m.var_covar(X, var)
print(c)

[[ 0.1786122  -0.01993074 -0.00513088  0.01567684]
 [-0.01993074  0.00237237  0.00043044 -0.00168614]
 [-0.00513088  0.00043044  0.00108079 -0.0008474 ]
 [ 0.01567684 -0.00168614 -0.0008474   0.00154512]]


In [14]:
# beräkna signifikans för de enskilda β-parametrarna
sig=[]
for a in range(d+1):
    sig.append(m.significance(a,b,c,S))
print(sig)
    

[-76.25702816954549, 224.5517087789827, 1382.6640099494732, -241.40224898279556]


In [15]:
for a in range(d+1):
    print(m.relevance(sig, a, n, d))
    

1.369442865878956e-146
2.279977868333894e-236
0.0
1.9192830886558436e-242


In [19]:
# print(stats.t.cdf(0.025, 200))
# print(stats.t.ppf(0.95,200))
print(stats.t.ppf(1-0.05/2,n-d-1))



1.972267532579456


In [17]:
# calculate confidence interval

for a in range(d+1):
    ci=m.confidence_interval(n,d,var,c,a)
    print(f"Konfidensintervallet för β{a} på 95%-nivån är mellan {b[a]+ci} och {b[a]-ci}")
    

Konfidensintervallet för β0 på 95%-nivån är mellan -2.554534655293283 och -2.5650516160478123
Konfidensintervallet för β1 på 95%-nivån är mellan 0.8693212249244441 och 0.8681091588053166
Konfidensintervallet för β2 på 95%-nivån är mellan 3.610827229208975 och 3.610009129744773
Konfidensintervallet för β3 på 95%-nivån är mellan -0.7531986370351633 och -0.754176809399355
